# 导入模块

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time 

import os
import sys
# 获取当前工作目录
# current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
# module_dir = os.path.join(current_dir, './Class_Model')
# sys.path.append(module_dir)
# from All_Class import All_Model, Week_Model
from Class_Model.All_Class import All_Model, Week_Model

In [2]:
# 实例化Week_Model类和All_Model类
all_models = All_Model()
week_models = Week_Model()
pd.set_option('display.max_columns', None)

# 定义数据库连接

In [3]:
def query(sql,
        host="rm-wz930e5269fur1ht1mo.mysql.rds.aliyuncs.com",
        user="wxz",
        password="5JRcY9SaiepVlIq7iuPo",
        database = '' ,
        port=3306
        ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入订单数据

In [4]:
sql1 = ''' -- 订单&风控信息  近10日数据   
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
,om.status
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
,top.total_describes,tor.decision_result,om.cancel_reason
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,cc.name as channel_name         -- 来源渠道
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name
,topay.total_freeze_fund_amount 
,om.buy_service_product,tso.status as service_status 
,om.order_method, tor.update_time, tomt.reason
from  db_digua_business.t_order  om 
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 
where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
-- 商家订单转移表
left join db_digua_business.t_order_merchant_transfer tomt on tomt.order_id=om.id
where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静'
,'陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')
-- and COALESCE(pa.name, '未知') not in ("1000单秘密计划","1000单秘密计划-无优惠","1000单曙光计划","线下门店3个月试行") 
-- AND tmer.shop_type !=2  -- 剔除商家  -- 剔除商家
and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -1 month )               -- 近15天数据

;
'''

df_order = query(sql1)
df_order.shape

(141395, 31)

In [5]:
sql3 = '''
SELECT id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status   FROM `db_credit`.risk_examine
'''
df_risk_examine = query(sql3)

# 合并数据

In [6]:
# df = df_order.merge(df_order_risk,left_on='订单id',right_on='order_id',how='left')
df = df_order.copy()
df.order_id.notnull().sum()

np.int64(141395)

# 处理日期

In [7]:

df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["下单日期"]
df["月份"]=df["下单日期"].dt.month
df.shape

(141395, 33)

In [8]:
df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')
df['拒绝理由'].value_counts()

拒绝理由
                                                                       69220
冰鉴火眸分<500命中强拒                                                          22036
命中强拒加入临时黑名单                                                             9682
年龄超过49岁或低于18岁                                                           3894
为法院限高被执行人                                                               1903
                                                                       ...  
命中云商分低于515强拒, 命中融安分低于680强拒, 评分等级低于D,命中拒绝                                   1
命中云商分低于515强拒(芝麻综合风险等级0豁免), 命中极信sc32007分强拒                                  1
命中被执行人或曝光台, 蚁盾分>=80, 评分等级低于D,命中拒绝                                          1
命中云商分低于496强拒, 命中策略240703_4强拒, 命中自有模型强拒, 评分等级低于D,命中拒绝                       1
命中策略240703_14强拒, 命中自有模型强拒, 命中蚁盾分>50, 命中融安分<700, 履约历史等级1, 履约历史等级=1强拒        1
Name: count, Length: 761, dtype: int64

# 处理备注信息

In [9]:
df = df[df['sku_attributes'].notnull()]

In [10]:

df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")

def getcolor(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="颜色":
            return color_list[j]["value"]
df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["sku_attributes"]),axis=1)
df["颜色"].value_counts()  

# %%
def getneicun(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="内存":
            return color_list[j]["value"]
df.loc[:,"内存"]=df.apply(lambda x:getneicun(x["sku_attributes"]),axis=1)
df["内存"].value_counts() 

df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
df["进件"].value_counts()

# %%
df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")

In [11]:
df['免审'] = np.where(df.decision_result.str.contains(pat='免人审',regex=False),1,0)
df['免审'].value_counts()

免审
0    135130
1      6248
Name: count, dtype: int64

# 归属渠道

In [12]:
def qudao_type(a,b,c):
    a = str(a)
    b = str(b)
    if "租物" in b:
        return "芝麻租物"
    elif "芝麻" in b:
        return "芝麻租物"
    elif "抖音" in b:
        return "抖音渠道"
    elif "搜索" in a:
        return "搜索渠道"
    elif "租物" in a:
        return "芝麻租物"
    elif "芝麻" in a:
        return "芝麻租物"   
    elif c == 1:
        return "芝麻租物" 
    elif "叮咚直播" in a:
        return "叮咚直播"
    elif "租瓜直播2号" in a:
        return "租瓜直播2号"
    elif "租瓜直播" in a:
        return "租瓜直播"
    elif "直播" in a:
        return "支付宝直播"
    elif "繁星" in a:
        return "繁星"
    elif "生活号" in a:
        return "生活号"
    elif "群" in a:
        return "支付宝社群"
    elif "付费灯火" in a:
        return "付费灯火"
    else :
        return a
    

# %%
df.loc[:,"归属渠道"]=df.apply(lambda x:qudao_type(x["来源渠道"],x["activity_name"],x["order_method"]),axis=1)
df["归属渠道"].value_counts()

归属渠道
芝麻租物                              66503
搜索渠道                              43249
单人聊天会话中的小程序消息卡片（分享）                8956
八派信息                               3480
其他渠道场景渠道。                          3131
小程序商家消息（服务提醒）                      2747
我的小程序入口                            2555
生活号                                2285
九州信息                               1291
付费流量(通过商家数字推广平台，灯火等投放的广告)          1201
付费灯火                                906
支付宝直播                               803
未知渠道                                750
支付宝社群                               717
支付宝 push 消息（同1014）                  644
付费邦道-首页                             524
抖音渠道                                300
付费邦道-苹果旗舰                           274
扫描二维码                               223
订阅消息                                200
流量扶持测试                              168
苹果SL23                              136
支付成功页                               124
线下易拉宝                                50
曙光计划                               

# 订单去重

In [13]:
dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df["状态编码"]=df["status2"].map(dict_status_code)


df.sort_values(by=["下单日期","状态编码"],inplace=True)


#删除重复单号
df.drop_duplicates(subset=["order_id"],inplace=True)
#删除身份证空值行
df.dropna(subset=["id_card_num"],axis=0,inplace=True)
#去刷单订单
df.drop(df[df['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单曙光计划"].index,inplace=True)
df.drop(df[df['activity_name']=="线下门店3个月试行"].index,inplace=True)
#删除身份证空值行
df.dropna(subset=["status2"],axis=0,inplace=True)


#删除重复订单
df.drop_duplicates(subset=["order_id"],inplace=True)
df.drop_duplicates(subset=["true_name","user_mobile","id_card_num","下单日期"],keep="last",inplace=True)


df.drop(df[df['true_name'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文"])].index,inplace=True)

# 定义状态

In [14]:
# 判断 前置拦截   机审强拒   出库前风控强拒
df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)


In [15]:
def reject_type(a,b,c,d,e,f,g,h,i,j):
    if h==1 and b=="未进件":
        return "前置拦截"
    elif i==1 and b=="进件":
        return "机审强拒"    
    #抖音渠道
    # elif h==1 and b=="进件":
    #     return "机审强拒"  
    elif  b=="进件" and c is not np.nan and '已退款' in e:
        return "人审拒绝"
    elif  b=="进件" and d is not np.nan:
        return "客户取消"
    elif  b=="进件" and f is not np.nan:
        return "无法联系"
    elif  b=="进件" and j==1 :
        return "出库前风控强拒"
    elif  b=="进件" and "待审核" in e:
        return "待审核"
    elif  b=="进件" and "待发货" in e:
        return "出库"
    elif  b=="进件" and "待收货" in e:
        return "出库"
    elif b=="进件" and "租赁中" in e:
        return "出库"
    elif  b=="进件" and "已完成" in e:
        return "出库"
    elif a is np.nan and b=="未进件":
        return "未进件"
    elif  b=="未进件":
        return "进件前取消"
    else :
        return e

df.loc[:,"审核状态"]=df.apply(lambda x: reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)#,x["无法联系原因"]
df["审核状态"].value_counts()  

审核状态
进件前取消      47838
前置拦截       21037
机审强拒        9903
出库          2813
人审拒绝        2475
客户取消        1427
已退款         1047
出库前风控强拒      504
无法联系         121
待审核           77
待归还            4
待商家收货          2
Name: count, dtype: int64

# 导入服务订单

In [16]:
from datetime import datetime  
today = datetime.today() 
t_date = today.date().strftime('%Y-%m-%d')
t_date

'2024-12-02'

In [17]:
df["下单时段"]=df["create_time"].astype(str).str[:14]

# %%
df_j = df[df["进件"]=="进件"]
df_j.shape

(18373, 54)

In [18]:
df["待审核"] = np.where(df["审核状态"]=='待审核',1,0)
df["前置拦截"] = np.where(df["审核状态"]=='前置拦截',1,0)
df["机审强拒"] = np.where(df["审核状态"]=='机审强拒',1,0)
df["人审拒绝"] = np.where(df["审核状态"]=='人审拒绝',1,0)
df["客户取消"] = np.where(df["审核状态"]=='客户取消',1,0)
df["出库前风控强拒"] = np.where(df["审核状态"]=='出库前风控强拒',1,0)
df["无法联系"] = np.where(df["审核状态"]=='无法联系',1,0)
df["是否进件"] = np.where(df["进件"]=='进件',1,0)
df["是否出库"] = np.where(df["status"].isin([2,3,4,5,6,8,15]),1,0)

df["进件前取消"] = np.where(df["审核状态"]=='进件前取消',1,0)

df['机审通过件'] = np.where((df['是否进件']==1)&(df['机审强拒']==0), 1, 0)
df['风控通过件'] = np.where((df['是否进件']==1)&(df['机审强拒']==0)&(df['人审拒绝']==0), 1, 0)

df['是否出库'] = np.where((df['人审拒绝']==0)&(df['客户取消']==0)&(df['无法联系']==0)&(df['待审核']==0)&(df['是否出库']==1), 1, 0)


# 剔除商家信息

In [19]:
#剔除商家数据只保留自营租机业务数据
df.drop(df[df['merchant_name']=="深圳优优大数据科技有限公司"].index,inplace=True)
df.drop(df[df['merchant_name']=="优优2店"].index,inplace=True)
df.drop(df[df['merchant_name']=="小豚租（代收）"].index,inplace=True)
df.drop(df[df['merchant_name']=="苏州蚁诺宝"].index,inplace=True)
df.drop(df[df['merchant_name']=="租着用电脑数码"].index,inplace=True)
df.drop(df[df['merchant_name']=="北京海鸟窝科技有限公司"].index,inplace=True)

df.drop(df[df['merchant_name']=="汇客好租"].index,inplace=True)
df.drop(df[df['merchant_name']=="澄心优租"].index,inplace=True)
df.drop(df[df['merchant_name']=="CPS渠道合作"].index,inplace=True)
df.drop(df[df['sku_attributes'].str.contains(pat='探路者',regex=False)==True].index,inplace=True)  
# 趣智数码  单
df.drop(df[df['merchant_name']=="趣智数码"].index,inplace=True)
df.drop(df[df['merchant_name']=="格木木二奢名品"].index,inplace=True)
df.drop(df[df['merchant_name']=="广州康基贸易有限公司"].index,inplace=True)
df.drop(df[df['merchant_name']=="线下小店"].index,inplace=True)
df.drop(df[df['merchant_name'].str.contains(pat='探路者',regex=False)==True].index,inplace=True) 

# 导入出库台账数据

In [20]:
#读取导入出库台账数据
f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2023&2024年台账.xlsx"
df_ck = pd.read_excel(f_path_ck,sheet_name="2023")   #,skiprows=0 header 
df_ck.shape

(23403, 19)

In [21]:
df_ck['溢价费订单'] = "A"+df_ck[df_ck['备注'].str.contains(pat='溢价费',regex=False)==True]['备注'].str.extract('(\d+.\d+)')
df_yijia = df_ck[df_ck['溢价费订单'].str.len()>=16][['溢价费订单',"已付金额"]]
df_yijia=df_yijia.rename(columns={'溢价费订单':"单号","已付金额":"溢价费"})
df_yijia_ck = pd.merge(df_ck,df_yijia,left_on="订单号", right_on="单号",how="left")
dfck = pd.merge(df_yijia_ck,df,left_on="订单号", right_on="order_number")

dfck.drop_duplicates(subset=["订单号"],inplace=True)
dfck.shape#.to_excel(f"C:/Users/Administrator/Desktop/报表模板/出库台账最终整理/11月出库补充1220.xlsx")

(2520, 88)

In [22]:
dfck[dfck["status2"]=="已退款"]

dfck.drop(dfck[dfck["status2"]=="已退款"].index,inplace=True)

dfck[dfck["status2"]=="已退款"]

# 删除 露营设备 出库 
dfck.drop(dfck[dfck["类目"]=="露营设备"].index,inplace=True)


In [23]:
#按下单日期看每日各渠道出库单数
df_weekday = pd.crosstab(dfck["下单日期"],dfck["归属渠道"],margins=True)

# 总体

In [24]:
#总体
# 周报所需数据
df_group_weekday = df.groupby(['下单日期']).agg({'order_id':'size','是否进件':'sum'})
df_group_weekday.rename(columns={'order_id':'去重订单数'},inplace=True )
df_group_weekday['出库'] = df_weekday['All'].astype(int)
weekly_sum = week_models.week_data_group(df_group_weekday, 4)
weekly_sum[['week_group', '去重订单数', '去重订单数环比', '是否进件', '是否进件环比', '预授权通过率', '预授权率环比差值', '出库', '出库环比', '进件出库率', '进件出库率环比差值', '订单出库率', '订单出库率环比差值']] 

,week_group,去重订单数,去重订单数环比,是否进件,是否进件环比,预授权通过率,预授权率环比差值,出库,出库环比,进件出库率,进件出库率环比差值,订单出库率,订单出库率环比差值
0,1108-1114,20325,nan%,3936,nan%,19.37%,nan%,566.0,nan%,14.38%,nan%,2.78%,nan%
1,1115-1121,19426,-4.42%,3930,-0.15%,20.23%,0.87%,572.0,1.06%,14.55%,0.17%,2.94%,0.16%
2,1122-1128,20883,7.50%,4580,16.54%,21.93%,1.70%,671.0,17.31%,14.65%,0.10%,3.21%,0.27%


# 搜索

In [25]:

df_ss = df[df["归属渠道"]=="搜索渠道"]

df_group_weekday_ss = df_ss.groupby(['下单日期']).agg({'order_id':'size','是否进件':'sum'})
df_group_weekday_ss.rename(columns={'order_id':'去重订单数'},inplace=True )
df_group_weekday_ss['出库'] = df_weekday['搜索渠道'].astype(int)
weekly_sum_ss = week_models.week_data_channel_group(df_group_weekday, df_group_weekday_ss, 4)
weekly_sum_ss[['week_group', '去重订单数', '订单占比', '是否进件', '进件占比', '出库', '出库占比', '进件出库转化率', '订单出库率']]

,week_group,去重订单数,订单占比,是否进件,进件占比,出库,出库占比,进件出库转化率,订单出库率
0,1108-1114,5426,26.70%,1545,39.25%,243.0,42.93%,15.73%,4.48%
1,1115-1121,5957,30.67%,1626,41.37%,244.0,42.66%,15.01%,4.10%
2,1122-1128,6873,32.91%,1945,42.47%,280.0,41.73%,14.40%,4.07%


# 单人聊天

In [26]:
df_dr=df[df["来源渠道"]=="单人聊天会话中的小程序消息卡片（分享）"]
df_group_weekday_dr = df_dr.groupby(['下单日期']).agg({'order_id':'size','是否进件':'sum'})
df_group_weekday_dr.rename(columns={'order_id':'去重订单数'},inplace=True )
df_group_weekday_dr['出库'] = df_weekday['单人聊天会话中的小程序消息卡片（分享）'].astype(int)

# 芝麻租物

In [27]:
df_zm=df[df["归属渠道"]=="芝麻租物"]

df_group_weekday_zm = df_zm.groupby(['下单日期']).agg({'order_id':'size','是否进件':'sum'})
df_group_weekday_zm.rename(columns={'order_id':'去重订单数'},inplace=True )
df_group_weekday_zm['出库'] = df_weekday['芝麻租物'].astype(int)
weekly_sum_zm = week_models.week_data_channel_group(df_group_weekday, df_group_weekday_zm, 4)
weekly_sum_zm[['week_group', '去重订单数', '订单占比', '是否进件', '进件占比', '出库', '出库占比', '进件出库转化率', '订单出库率']]

,week_group,去重订单数,订单占比,是否进件,进件占比,出库,出库占比,进件出库转化率,订单出库率
0,1108-1114,9820,48.31%,1603,40.73%,254.0,44.88%,15.85%,2.59%
1,1115-1121,8413,43.31%,1466,37.30%,243.0,42.48%,16.58%,2.89%
2,1122-1128,9727,46.58%,1861,40.63%,301.0,44.86%,16.17%,3.09%


# 抖音

In [28]:
df_dy=df[df["归属渠道"]=="抖音渠道"]

df_group_weekday_dy = df_dy.groupby(['下单日期']).agg({'order_id':'size','进件前取消':'sum'})
df_group_weekday_dy.rename(columns={'order_id':'去重订单数', '进件前取消': '是否进件'},inplace=True )
df_group_weekday_dy['出库'] = df_weekday['抖音渠道'].astype(int)

# 总体剔除直播数据

In [29]:
weekly_sum_tc = df_group_weekday - df_group_weekday_dr
weekly_sum_tc['出库'] =  df_group_weekday['出库'] - df_group_weekday_dr['出库'] - df_group_weekday_dy['出库']
weekly_sum_tc_group = week_models.week_data_channel_group(df_group_weekday, weekly_sum_tc, 4)
weekly_sum_tc_group[['week_group', '去重订单数', '是否进件', '出库', '预授权通过率', '进件出库转化率', '订单出库率']]

,week_group,去重订单数,是否进件,出库,预授权通过率,进件出库转化率,订单出库率
0,1108-1114,18948,3647,548.0,19.25%,15.03%,2.89%
1,1115-1121,17856,3594,543.0,20.13%,15.11%,3.04%
2,1122-1128,19167,4206,648.0,21.94%,15.41%,3.38%


# 免审

In [30]:

df_ms = df[df["decision_result"].str.contains(pat="免人审",regex=False)==True]
df_group_ms = df_ms.groupby(['下单日期']).agg({'是否进件':'sum', '是否出库':'sum'})
df_group_ms.rename(columns={'是否进件': '免审进件', '是否出库': '免审出库'}, inplace=True)
df_group_ms['总体进件'] = df_group_weekday['是否进件'][:-1]
df_group_ms['芝麻租物进件'] = df_group_weekday_zm['是否进件'][:-1]

# 各渠道每日免审出库单数统计
df_group_ms['总体出库'] = df_group_weekday['出库'][:-1]
df_group_ms['芝麻租物出库'] = df_group_weekday_zm['出库'][:-1]

# 免审出库
dfdfck=dfck[dfck["decision_result"].str.contains(pat="免人审",regex=False)==True]
dfdfck_ms = pd.crosstab(dfdfck["下单日期"],dfdfck["归属渠道"],margins=True).reset_index()[['下单日期', '芝麻租物','搜索渠道']][:-1]
dfdfck_ms['下单日期'] = pd.to_datetime(dfdfck_ms['下单日期'])
dfdfck_ms.rename(columns={'芝麻租物': '芝麻租物免审出库', '搜索渠道': '搜索免审出库'}, inplace=True)
dfms = pd.crosstab(df_ms["下单日期"],df_ms["归属渠道"],margins=True).reset_index()[:-1]
dfms['下单日期']  = pd.to_datetime(dfms['下单日期'])
dfms = dfms[['下单日期', "芝麻租物","搜索渠道"]]

dfms.rename(columns={'芝麻租物': '芝麻租物免审进件', '搜索渠道': '搜索免审进件'}, inplace=True)
df_group_weekday_ms = pd.merge(df_group_ms, dfms, on='下单日期', how='outer')[:-1]

df_group_weekday_ms = pd.merge(df_group_weekday_ms, dfdfck_ms, on='下单日期', how='outer')
df_group_weekday_ms.set_index('下单日期', inplace=True)

weekly_sum_ms = week_models.week_data_m_group(df_group_weekday_ms, 7, '免审')
weekly_sum_ms['搜索免审进件占比'] = weekly_sum_ms['搜索免审进件']/weekly_sum_ss['是否进件'] * 100
weekly_sum_ms['搜索免审进件占比'] = weekly_sum_ms['搜索免审进件占比'].apply(lambda x: f'{x:.2f}%')

weekly_sum_ms['搜索免审出库占比'] = weekly_sum_ms['搜索免审出库']/weekly_sum_ss['出库'] * 100
weekly_sum_ms['搜索免审出库占比'] = weekly_sum_ms['搜索免审出库占比'].apply(lambda x: f'{x:.2f}%')

weekly_sum_ms[['week_group', '总体进件',  '芝麻租物进件', '免审进件', '芝麻租物免审进件', '搜索免审进件', '免审进件占比', '芝麻租物免审进件占比', '搜索免审进件占比', '总体出库', '芝麻租物出库', '免审出库', '芝麻租物免审出库', '搜索免审出库', '免审转化率', '免审出库占比', '芝麻租物免审出库占比', '搜索免审出库占比']]

,week_group,总体进件,芝麻租物进件,免审进件,芝麻租物免审进件,搜索免审进件,免审进件占比,芝麻租物免审进件占比,搜索免审进件占比,总体出库,芝麻租物出库,免审出库,芝麻租物免审出库,搜索免审出库,免审转化率,免审出库占比,芝麻租物免审出库占比,搜索免审出库占比
0,1104-1110,3759.0,1529.0,956,494,337,25.43%,32.31%,21.81%,530.0,229.0,494,216,237,51.67%,93.21%,94.32%,97.53%
1,1111-1117,3956.0,1593.0,1050,563,323,26.54%,35.34%,19.86%,528.0,247.0,496,239,201,47.24%,93.94%,96.76%,82.38%
2,1118-1124,4061.0,1543.0,1035,538,385,25.49%,34.87%,19.79%,637.0,266.0,591,256,277,57.10%,92.78%,96.24%,98.93%
3,1125-1201,4762.0,1956.0,1213,627,427,25.47%,32.06%,nan%,649.0,297.0,633,293,271,52.18%,97.53%,98.65%,nan%


# 免审订单转化

In [31]:
df_msdd = df[df["decision_result"].str.contains(pat="免人审",regex=False)==True]
df_group_mszh = df_msdd.groupby(['下单日期']).agg({'是否进件':'sum', '机审强拒': 'sum', '机审通过件': 'sum',  '人审拒绝':'sum', '风控通过件': 'sum', '客户取消':'sum','无法联系':'sum','出库前风控强拒':'sum','待审核':'sum','是否出库':'sum'})
df_group_mszh.rename(columns={'是否出库': '出库'}, inplace=True)
# df_ms = df_group_mszh[["是否进件","人审拒绝","客户取消","是否出库","待审核",'出库前风控强拒','无法联系']]
weekly_sum_mszh = week_models.week_data_m_group(df_group_mszh, 7, '免审订单')
weekly_sum_mszh[['week_group', '是否进件', '人审拒绝', '客户取消', '出库', '待审核', '进件出库率', '取消率', '人审拒绝率', '出库前风控强拒', '出库前强拒比例', '无法联系', '无法联系占比']]

,week_group,是否进件,人审拒绝,客户取消,出库,待审核,进件出库率,取消率,人审拒绝率,出库前风控强拒,出库前强拒比例,无法联系,无法联系占比
0,1104-1110,956,94,221,494,0,51.67%,23.12%,9.83%,88,9.21%,15,1.57%
1,1111-1117,1050,93,279,496,0,47.24%,26.57%,8.86%,103,9.81%,12,1.14%
2,1118-1124,1035,75,227,591,0,57.10%,21.93%,7.25%,76,7.34%,10,0.97%
3,1125-1201,1213,108,282,633,29,52.18%,23.25%,8.90%,96,7.91%,5,0.41%


# 免押订单转化

In [32]:
df2 = df.copy()
df2.drop(df2[df2['merchant_name']=="小蚂蚁租机"].index,inplace=True)

In [33]:
df_mydd1 = df[df["押金类型"]=="全免押"]
df_mydd2 = df2[df2["押金类型"]=="全免押"]
df_group_mydd1 = df_mydd1.groupby(['下单日期']).agg({'是否进件':'sum'})
df_group_mydd2 = df_mydd2.groupby(['下单日期']).agg({'机审强拒':'sum', '机审通过件':'sum', '人审拒绝':'sum', '风控通过件':'sum', '客户取消':'sum', '无法联系': 'sum', '待审核':'sum', '出库前风控强拒': 'sum'})
# df_group_mydd1.rename(columns={'order_id':'去重订单数'},inplace=True )
# 获取小蚂蚁租机的数据
df_xmy = all_models.xmy(df_mydd1, df_risk_examine, '下单日期')
df_group_mydd = df_group_mydd1.merge(df_group_mydd2, on='下单日期', how='inner').merge(df_xmy, on='下单日期', how='left')
df_group_mydd = df_group_mydd.fillna(0)
df_group_mydd["机审强拒"] = df_group_mydd["机审强拒"]+df_group_mydd['小蚂蚁机审强拒']
df_group_mydd['总进件'] = df_group_weekday['是否进件']
df_group_mydd["机审通过件"] = df_group_mydd["机审通过件"]+ df_group_mydd['小蚂蚁人审拒绝'] + df_group_mydd['order_risk_ex']
df_group_mydd["风控通过件"] = df_group_mydd["风控通过件"] + df_group_mydd['order_risk_ex']
myck = pd.crosstab(dfck["下单日期"],dfck["押金类型"],margins=True)
df_group_mydd['出库'] = myck['全免押']

weekly_sum_mydd = week_models.week_data_m_group(df_group_mydd, 7, '免审订单')
weekly_sum_mydd['免押进件占比'] = weekly_sum_mydd['是否进件']/weekly_sum_mydd['总进件'] * 100
weekly_sum_mydd['免押进件占比'] = weekly_sum_mydd['免押进件占比'].apply(lambda x: f'{x:.2f}%')
weekly_sum_mydd[['week_group', '总进件', '免押进件占比', '是否进件', '机审强拒', '强拒比例', '机审通过件', '人审拒绝', '风控通过件', '风控通过率', '客户取消', '待审核', "出库", '进件出库率', '取消率', '人审转化率', '人审拒绝率']]

,week_group,总进件,免押进件占比,是否进件,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,待审核,出库,进件出库率,取消率,人审转化率,人审拒绝率
0,1104-1110,3759,89.70%,3372,2291.0,67.94%,1081.0,130,951.0,28.20%,245,0,530.0,15.72%,7.27%,49.03%,3.86%
1,1111-1117,3956,88.50%,3501,2350.0,67.12%,1151.0,117,1034.0,29.53%,297,0,525.0,15.00%,8.48%,45.61%,3.34%
2,1118-1124,4061,89.21%,3623,2440.0,67.35%,1183.0,110,1071.0,29.56%,256,0,631.0,17.42%,7.07%,53.34%,3.04%
3,1125-1201,4762,90.05%,4288,2899.0,67.61%,1389.0,171,1218.0,28.40%,314,37,645.0,15.04%,7.32%,46.44%,3.99%


# 非免押订单转化

In [34]:
df_fmydd1 = df[df["押金类型"]=="部分免押"]
df_fmydd2 = df2[df2["押金类型"]=="部分免押"]
df_group_fmydd1 = df_fmydd1.groupby(['下单日期']).agg({'是否进件':'sum'})
df_group_fmydd2 = df_fmydd2.groupby(['下单日期']).agg({'机审强拒':'sum', '机审通过件':'sum', '人审拒绝':'sum', '风控通过件':'sum', '客户取消':'sum', '无法联系': 'sum', '待审核':'sum', '出库前风控强拒': 'sum'})
# df_group_mydd1.rename(columns={'order_id':'去重订单数'},inplace=True )
# 获取小蚂蚁租机的数据
df_xmy = all_models.xmy(df_fmydd1, df_risk_examine, '下单日期')
df_group_fmydd = df_group_fmydd1.merge(df_group_fmydd2, on='下单日期', how='inner').merge(df_xmy, on='下单日期', how='left')
df_group_fmydd = df_group_fmydd.fillna(0)
df_group_fmydd["机审强拒"] = df_group_fmydd["机审强拒"]+df_group_fmydd['小蚂蚁机审强拒']
df_group_fmydd['总进件'] = df_group_weekday['是否进件']
df_group_fmydd["机审通过件"] = df_group_fmydd["机审通过件"]+ df_group_fmydd['小蚂蚁人审拒绝'] + df_group_fmydd['order_risk_ex']
df_group_fmydd["风控通过件"] = df_group_fmydd["风控通过件"] + df_group_fmydd['order_risk_ex']
df_group_fmydd["待审核"] = 0
df_group_fmydd["出库"] = 0
weekly_sum_fmydd = week_models.week_data_m_group(df_group_fmydd, 7, '免审订单')
weekly_sum_fmydd['非免押进件占比'] = weekly_sum_fmydd['是否进件']/weekly_sum_fmydd['总进件'] * 100
weekly_sum_fmydd['非免押进件占比'] = weekly_sum_fmydd['非免押进件占比'].apply(lambda x: f'{x:.2f}%')
weekly_sum_fmydd[['week_group', '总进件', '非免押进件占比', '是否进件', '机审强拒', '强拒比例', '机审通过件', '人审拒绝', '风控通过件', '风控通过率', '客户取消', '待审核', "出库", '进件出库率', '取消率', '人审转化率', '人审拒绝率']]

,week_group,总进件,非免押进件占比,是否进件,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,待审核,出库,进件出库率,取消率,人审转化率,人审拒绝率
0,1104-1110,3759,10.30%,387,384.0,99.22%,3.0,0,3.0,0.78%,1,0,0,0.00%,0.26%,0.00%,0.00%
1,1111-1117,3956,11.50%,455,448.0,98.46%,7.0,0,7.0,1.54%,1,0,0,0.00%,0.22%,0.00%,0.00%
2,1118-1124,4061,10.79%,438,427.0,97.49%,11.0,1,10.0,2.28%,1,0,0,0.00%,0.23%,0.00%,0.23%
3,1125-1201,4762,9.95%,474,452.0,95.36%,22.0,5,17.0,3.59%,1,0,0,0.00%,0.21%,0.00%,1.05%
